In [17]:
import torch
import torch.optim as optim
from torchvision import datasets, transforms
import albumentations as A
from torchsummary import summary
import torch.nn.functional as F


from models.model_assignment6 import myCNN

CIFAR10 mean = (0.49139968, 0.48215827 ,0.44653124)
CIFAR10 std = (0.24703233, 0.24348505, 0.26158768)

In [2]:
from albumentations.pytorch import ToTensorV2

In [7]:
train_tfms = A.Compose(
    [
        A.HorizontalFlip(),
        A.ShiftScaleRotate(),
        A.CoarseDropout(max_holes=1, max_height=16, max_width=16, min_holes=1, min_height=16, min_width=16, 
        fill_value=[0.49139968, 0.48215827 ,0.44653124], mask_fill_value=None),
        A.Normalize(mean=(0.49139968, 0.48215827 ,0.44653124), std=(0.24703233, 0.24348505, 0.26158768)),
        ToTensorV2(),
    ]
)

test_tfms = A.Compose([
    A.Normalize(mean=(0.49139968, 0.48215827 ,0.44653124), std=(0.24703233, 0.24348505, 0.26158768)),
    ToTensorV2(),
    
])

In [22]:
train_data = datasets.CIFAR10("/home/raghuc/work/data/", train=True, download=True, transform=train_tfms)
test_data = datasets.CIFAR10("/home/raghuc/work/data/", train=False, download=True, transform=test_tfms)

TypeError: list indices must be integers or slices, not str

In [9]:
SEED = 1
use_cuda = torch.cuda.is_available()
torch.manual_seed(SEED)
if use_cuda:
    torch.cuda.manual_seed(SEED)

dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if use_cuda else dict(shuffle=True, batch_size=64)

train_loader = torch.utils.data.DataLoader(train_data, **dataloader_args)
test_loader = torch.utils.data.DataLoader(test_data, **dataloader_args)

In [10]:
Net = myCNN()

In [15]:
device = torch.device("cuda" if use_cuda else "cpu")
model = Net.to(device)
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
           Dropout-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 64, 15, 15]          18,432
       BatchNorm2d-6           [-1, 64, 15, 15]             128
              ReLU-7           [-1, 64, 15, 15]               0
           Dropout-8           [-1, 64, 15, 15]               0
            Conv2d-9          [-1, 256, 11, 11]         147,456
      BatchNorm2d-10          [-1, 256, 11, 11]             512
             ReLU-11          [-1, 256, 11, 11]               0
          Dropout-12          [-1, 256, 11, 11]               0
           Conv2d-13            [-1, 256, 9, 9]           2,304
           Conv2d-14             [-1, 6

In [16]:
from tqdm import tqdm
train_losses = []
test_losses = []
train_acc = []
test_acc = []

In [18]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0

    for batch_idx, (data, tgt) in enumerate(pbar):
        data, tgt = data.to(device), tgt.to(device)

        optimizer.zero_grad()
        y_pred = model(data)
        loss = F.nll_loss(y_pred, tgt)
        train_losses.append(loss)

        loss.backward()
        optimizer.step()

        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(tgt.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)

In [19]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0.0
    correct = 0

    with torch.no_grad():
        for data, tgt in test_loader:
            data, tgt = data.to(device), tgt.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, tgt, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(tgt.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

    test_acc.append(100*correct/len(test_loader.dataset))


In [21]:
from torch.optim.lr_scheduler import StepLR

model = Net.to(device)
optimizer = optim.SGD(model.parameters(), lr = 0.05, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.25)


EPOCHS = 25
for epoch in range(EPOCHS):
    print("EPOCH: ", epoch)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)

EPOCH:  0


  0%|          | 0/391 [00:00<?, ?it/s]


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/raghuc/miniconda3/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/raghuc/miniconda3/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/raghuc/miniconda3/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/raghuc/miniconda3/lib/python3.10/site-packages/torchvision/datasets/cifar.py", line 118, in __getitem__
    img = self.transform(img)
  File "/home/raghuc/miniconda3/lib/python3.10/site-packages/albumentations/core/composition.py", line 188, in __call__
    raise KeyError("You have to pass data to augmentations as named arguments, for example: aug(image=image)")
K